In [1]:
import pandas as pd
import numpy as np
import mibian
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from datetime import datetime
import datetime

### Training Model

In [2]:
def pre_train_process(df,startIdx,windowLength,isNormalized):
    tmp = df.copy()
    tmp.reset_index(inplace=True)
    train_start_index = startIdx
    train_window_length = windowLength
    train_end_index = train_start_index+train_window_length-1
    tmp = tmp.iloc[train_start_index:train_end_index]
    ## normalize the option price change and future price change
    if(isNormalized==False):
        tmp['opt_price_chg'] = tmp['opt_price'].diff()
        tmp['fut_price_chg'] = tmp['fut_price'].diff()
        tmp = tmp[tmp['fut_price_chg']!=0]
    else:
        tmp['opt_price_chg'] = tmp['opt_price'] / tmp['opt_price'].shift()
        tmp['fut_price_chg'] = tmp['fut_price'] / tmp['fut_price'].shift()
        tmp = tmp[tmp['fut_price_chg']!=1]
    tmp = tmp[['ts','opt_price_chg','fut_price_chg','delta','vega','days_to_expiry','fut_price']]
    tmp.reset_index(drop=True,inplace=True)
    train_set = pd.DataFrame()
    coeff = tmp['vega']*tmp['fut_price_chg']/tmp['fut_price']/np.sqrt(tmp['days_to_expiry']/252)
    train_set['y'] = (tmp['opt_price_chg'] - tmp['delta']*tmp['fut_price_chg'])
    train_set['x1'] = coeff*tmp['delta']
    train_set['x2'] = coeff*tmp['delta']**2
    train_set.drop(index=0,inplace=True)
    return train_set

In [3]:
def train_model(train_set):
    x = sm.add_constant(train_set.iloc[:,1:])
    y = train_set['y']
    model = sm.OLS(y,x)
    result = model.fit()
    return result

In [4]:
def predict_MV_delta(df,startIdx,windowLength,result):
    tmp = df.copy()
    test_start_index = startIdx
    test_window_length = windowLength
    test_end_index = test_start_index + test_window_length-1
    tmp = tmp.iloc[test_start_index:test_end_index]
    tmp = tmp[['ts','delta','vega','days_to_expiry','fut_price']]
    b0 = result.params[0]
    b1 = result.params[1]
    b2 = result.params[2]
    tmp['y_hat'] = b0+b1*tmp['delta']+b2*tmp['delta']**2
    tmp['MV_delta'] = tmp['delta']+tmp['y_hat']*tmp['vega']/tmp['fut_price']/np.sqrt(tmp['days_to_expiry']/252)
    tmp.reset_index(inplace=True)
    return tmp

### Read in data

In [5]:
Calls = pd.read_csv('nifty_311221_190122_1min_calls_py_vollib.csv')
contractList = Calls['opt_symbol'].unique()
#contractList

In [6]:
def generate_MV_delta(contractName,trainStartIdx,trainLength,isNormalized,predictMVdeltaLength):
    contract = Calls[Calls['opt_symbol']==contractName]
    train_set = pre_train_process(contract,trainStartIdx,trainLength,isNormalized)
    result = train_model(train_set)
    print("result rsquared: ",result.rsquared)
    predict_set = predict_MV_delta(contract,trainStartIdx+trainLength,predictMVdeltaLength,result)
    return predict_set

### generate_MV_delta2 takes option name, predicted date, wheather to normalize data and rsquare threshold to genearte predicted MV_delta

In [7]:
def generate_MV_delta2(contractName,Date,isNormalized,threshold):
    contract = Calls[Calls['opt_symbol']==contractName].copy()
    contract.reset_index(inplace=True,drop=True)
    idx = contract.index[contract['ts']<=Date].tolist()[-1]
    #print("available data rows:", idx)
    i = 200
    rsquare = 0
    while(rsquare<threshold and i <= idx):
        train_set = pre_train_process(contract,idx-i,i,isNormalized)
        result = train_model(train_set)
        rsquare = result.rsquared
        i += 10
    if rsquare < threshold:
        print('WARNING: not enough data to make the regression model stable!')
    #print('i: ',i)
    #print("result rsquared: ",result.rsquared)
    #print(result.summary())
    predict_set = predict_MV_delta(contract,idx,2,result)
    return predict_set

### Back Test

In [ ]:
contractName = 'NIFTY22JAN17750CE'
contract = Calls[Calls['opt_symbol']==contractName]
#contract.reset_index(inplace=True,drop=True)
#contract

In [8]:
def generate_deltas(contractName,startDate,nDays,threshold):
    date = startDate
    res = pd.DataFrame(columns=['index','ts','delta','vega','days_to_expiry','fut_price','y_hat','MV_delta'])
    for i in range(0,nDays):
        predicted = generate_MV_delta2(contractName,date.strftime("%Y-%m-%d, %H:%M:%S"),True,threshold)
        res = res.append(predicted)
        date = date+datetime.timedelta(1)
    return res

In [9]:
def calculate_PnL(df):
    PnLs = pd.DataFrame(columns=['Delta_PnL','MV_delta_PnL'])
    for index, row in df.iterrows():
        if(index>0):
            delta_PnL = (row['fut_price'] - df.iloc[index-1]['fut_price'])*df.iloc[index-1]['delta']
            MV_delta_PnL = (row['fut_price'] - df.iloc[index-1]['fut_price'])*df.iloc[index-1]['MV_delta']
            df_tmp = pd.DataFrame({'Delta_PnL':[delta_PnL],'MV_delta_PnL':[MV_delta_PnL]})
            PnLs = PnLs.append(df_tmp)
    return PnLs

In [10]:
contractName = 'NIFTY22JAN18750CE'
startDate = datetime.datetime(2022,1,3,15,25)
res = generate_deltas(contractName,startDate,7,0.9)
res = res.reset_index()
res

,level_0,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,0,373,2022-01-03 15:29:00,0.056058,339.884683,24.001042,17692.35,0.516548,0.088212
1,0,748,2022-01-04 15:29:00,0.075719,336.402898,23.001042,17856.00,-0.371757,0.052537
2,0,1123,2022-01-05 15:29:00,0.093009,330.690544,22.001042,17946.20,-0.465342,0.063988
3,0,1498,2022-01-06 15:29:00,0.067604,320.141028,21.001042,17795.00,-0.104558,0.061088
4,0,1873,2022-01-07 15:29:00,0.068333,314.256085,20.001042,17852.20,-0.125201,0.060510
5,0,1873,2022-01-07 15:29:00,0.068333,314.256085,20.001042,17852.20,-0.125201,0.060510
6,0,1873,2022-01-07 15:29:00,0.068333,314.256085,20.001042,17852.20,-0.125201,0.060510


In [11]:
pnls = calculate_PnL(res)
pnls

,Delta_PnL,MV_delta_PnL
0,9.173817,14.435903
0,6.829873,4.738809
0,-14.062886,-9.675035
0,3.866939,3.494224
0,0.000000,0.000000
0,0.000000,0.000000


In [12]:
contractName = 'NIFTY22JAN17750CE'
startDate = datetime.datetime(2022,1,3,15,25)
res = generate_deltas(contractName,startDate,7,0.9)
res = res.reset_index()
res

,level_0,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,0,749,2022-01-03 15:29:00,0.473432,330.955810,24.001042,17692.35,-2.222898,0.338694
1,0,1124,2022-01-04 15:29:00,0.571558,326.410570,23.001042,17856.00,-8.395706,0.063558
2,0,1499,2022-01-05 15:29:00,0.617905,318.141385,22.001042,17946.20,-2.256829,0.482503
3,0,1874,2022-01-06 15:29:00,0.533106,308.789847,21.001042,17795.00,-5.251940,0.217414
4,0,2249,2022-01-07 15:29:00,0.569365,304.387541,20.001042,17852.20,-4.742629,0.282335
5,0,2249,2022-01-07 15:29:00,0.569365,304.387541,20.001042,17852.20,-4.742629,0.282335
6,0,2249,2022-01-07 15:29:00,0.569365,304.387541,20.001042,17852.20,-4.742629,0.282335


In [13]:
pnls = calculate_PnL(res)
pnls

,Delta_PnL,MV_delta_PnL
0,77.477118,55.427290
0,51.554567,5.732962
0,-93.427268,-72.954494
0,30.493686,12.436059
0,0.000000,0.000000
0,0.000000,0.000000


In [20]:
contractName = 'NIFTY22JAN16750CE'
startDate = datetime.datetime(2022,1,10,15,25)
res = generate_deltas(contractName,startDate,7,0.9)
res = res.reset_index()
res

,level_0,index,ts,delta,vega,days_to_expiry,fut_price,y_hat,MV_delta
0,0,1644,2022-01-10 14:23:00,0.997786,275.823551,17.046875,17999.70,-0.051359,0.994760
1,0,1812,2022-01-11 15:29:00,0.926793,252.524598,16.001042,18085.05,1.338143,1.000943
2,0,1875,2022-01-12 14:34:00,0.990598,256.651302,15.039236,18217.05,-1.574446,0.899799
3,0,2030,2022-01-13 14:58:00,0.997606,249.677850,14.022569,18253.55,-4.618300,0.729812
4,0,2038,2022-01-14 13:34:00,0.995294,240.484369,13.080903,18240.30,-4.790924,0.718054
5,0,2038,2022-01-14 13:34:00,0.995294,240.484369,13.080903,18240.30,-4.790924,0.718054
6,0,2038,2022-01-14 13:34:00,0.995294,240.484369,13.080903,18240.30,-4.790924,0.718054


In [21]:
pnls = calculate_PnL(res)
pnls

,Delta_PnL,MV_delta_PnL
0,85.161046,84.902780
0,122.336612,132.124448
0,36.156821,32.842662
0,-13.218283,-9.670009
0,0.000000,0.000000
0,0.000000,0.000000
